In [1]:
print("Ok")

Ok


In [36]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.document_loaders import PyPDFLoader, DirectoryLoader


In [37]:
PINECONE_API_KEY = "pcsk_36Xv9K_A4djaiHKUaQmUxDsJpQYgVxnGyC3Mc1zW9WXqMwX484TtqUsEv1f6MVrLSCci9w"
PINECONE_API_ENV = "us-east-1-aws"

In [38]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [9]:
extracted_data=load_pdf_file(data='data/')


In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 665


In [29]:
text_chunks

[Document(metadata={'source': 'data\\Physical_Activity_Guidelines_2nd_edition.pdf', 'page': 0}, page_content='Physical Activity \nGuidelines for Americans \n2nd edition'),
 Document(metadata={'source': 'data\\Physical_Activity_Guidelines_2nd_edition.pdf', 'page': 1}, page_content='MESSAGE FROM THE SECRETARY\nRegular physical activity is one of the most important things people can do to improve their health. Moving \nmore and sitting less have tremendous benefits for everyone, regardless of age, sex, race, ethnicity, or current \nfitness level. Individuals with a chronic disease or a disability benefit from regular physical activity, as do \nwomen who are pregnant. The scientific evidence continues to build—physical activity is linked with even more'),
 Document(metadata={'source': 'data\\Physical_Activity_Guidelines_2nd_edition.pdf', 'page': 1}, page_content='positive health outcomes than we previously thought. And, even better, benefits can start accumulating with \nsmall amounts of, 

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()


C:\Users\Admin\AppData\Local\Temp\ipykernel_20040\3158813755.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Admin\anaconda3\envs\tony\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\Admin\anaconda3\envs\tony\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_

In [15]:

query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
#query_result

In [39]:
from pinecone import Pinecone

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define your index
index_name = "medical-chatbot"

# Connect to an existing index
index = pc.Index(index_name)
print(f"Connected to Pinecone index: {index_name}")


Connected to Pinecone index: medical-chatbot


In [34]:
from langchain.embeddings import HuggingFaceEmbeddings

# Download the embeddings model
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings for text chunks
embedded_data = [
    {
        "id": str(i),  # Unique ID for each vector
        "values": embeddings.embed_query(chunk.page_content),
        "metadata": {"content": chunk.page_content}
    }
    for i, chunk in enumerate(text_chunks)
]
print(f"Generated embeddings for {len(embedded_data)} chunks.")

Generated embeddings for 665 chunks.


In [44]:
# Upsert data to Pinecone index
namespace = "medical-data"  # Optional namespace to organize data

index.upsert(vectors=embedded_data, namespace=namespace)
print(f"Uploaded {len(embedded_data)} vectors to namespace '{namespace}' in index '{index_name}'.")

Uploaded 665 vectors to namespace 'medical-data' in index 'medical-chatbot'.


In [46]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [47]:
# Create a PromptTemplate object
PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Step 3: Create an LLMChain
chain_type_kwargs = {"prompt" : PROMPT}

In [49]:
# Initialize LLM with CTransformers
llm = CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",  # Path to the Llama 2 model file
    model_type="llama",  # Specify model type
    config={
        "max_new_tokens": 512,  # Maximum tokens to generate
        "temperature": 0.8      # Sampling temperature
    }
)

In [ ]:
import os
import pinecone
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

# Set Pinecone API key and environment
PINECONE_API_KEY = "pcsk_36Xv9K_A4djaiHKUaQmUxDsJpQYgVxnGyC3Mc1zW9WXqMwX484TtqUsEv1f6MVrLSCci9w"
PINECONE_API_ENV = "us-east-1-aws"

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

# Step 1: Define a retriever using Pinecone's from_existing_index method
retriever = Pinecone.from_existing_index(
    index_name=index_name,  # Pinecone index name
    embedding=embeddings.embed_query,  # Embedding function
    namespace=namespace  # Namespace for the index
).as_retriever(search_kwargs={"k": 3})  # Retrieve top 3 documents

# Step 2: Define the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # Use your initialized CTransformers LLM
    chain_type="stuff",  # Combine all retrieved documents into a single context
    retriever=retriever,  # Use the retriever for document retrieval
    return_source_documents=True,  # Return source documents along with the answer
    chain_type_kwargs={"prompt": PROMPT}  # Use your defined prompt template
)

# Step 3: Query the RetrievalQA chain
query_text = "How much physical activity do adults need each week?"
result = qa_chain({"query": query_text})

# Step 4: Display the answer and source documents
print("Answer:", result["result"])

# Display the source documents (optional)
print("\nSource Documents:")
for doc in result["source_documents"]:
    print(doc["content"])


TypeError: __init__() got an unexpected keyword argument 'api_key'